In [1]:
from google.colab import files

a = files.upload()

Saving BananaVIS.csv to BananaVIS.csv


In [2]:
import tensorflow as tf
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
from numpy import diff
import random
%matplotlib inline
import shutil
import tensorflow.contrib.learn as tflearn
import tensorflow.contrib.layers as tflayers
from tensorflow.contrib.learn.python.learn import learn_runner
import tensorflow.contrib.metrics as metrics
import tensorflow.contrib.rnn as rnn
from sklearn.model_selection import train_test_split
!git clone https://github.com/EBjerrum/Deep-Chemometrics.git
  
def dataaugment(x, betashift = 0.05, slopeshift = 0.05,multishift = 0.05):
    #Shift of baseline
    #calculate arrays
    beta = np.random.random(size=(x.shape[0],1))*2*betashift-betashift
    slope = np.random.random(size=(x.shape[0],1))*2*slopeshift-slopeshift + 1
    #Calculate relative position
    axis = np.array(range(x.shape[1]))/float(x.shape[1])
    #Calculate offset to be added
    offset = slope*(axis) + beta - axis - slope/2. + 0.5

    #Multiplicative
    multi = np.random.random(size=(x.shape[0],1))*2*multishift-multishift + 1

    x = multi*x + offset

    return x
  
data = pd.read_csv('BananaVIS.csv')

spectrums= np.array(data.values[:, 0:288],dtype = np.int32)
categories=np.array(data.values[:,-1],dtype=np.int32)
portion = int(4/5 *spectrums.shape[0])

#Split train and test set
x_train, x_test, y_train,y_test = train_test_split(spectrums[:portion],categories[:portion],test_size=0.2,random_state=42)

x_check = spectrums[portion:]
y_check = categories[portion:]

%cd Deep-Chemometrics

from ChemUtils import GlobalStandardScaler

xscaler = GlobalStandardScaler()
x_train = xscaler.fit_transform(x_train) 
x_test = xscaler.transform(x_test) 
x_check = xscaler.transform(x_check)

yscaler = GlobalStandardScaler()
y_train = yscaler.fit_transform(y_train)
y_test = yscaler.transform(y_test)
y_check = yscaler.transform(y_check)

#EMSC rescaling
from ChemUtils import EmscScaler
emsc = EmscScaler()
emsc.fit(x_train)
X_train_emsc = emsc.transform((x_train))
X_test_emsc = emsc.transform(x_test)
X_check_emsc = emsc.transform(x_check)

#Data Augment a single spectrum

#First Spectrum
X = x_test[0:1]
#Repeating the spectrum 10x
X = np.repeat(X, repeats=10, axis=0)
#Augment (Large pertubations for illustration)
X_aug = dataaugment(X,betashift = 0.5, slopeshift = 0.5,multishift = 0.5)
shift = np.std(x_train)*0.1
X_train_aug = np.repeat(X_train_emsc, repeats=1000, axis=0)
X_train_aug = dataaugment(X_train_aug, betashift = shift, slopeshift = 0.05, multishift = shift)
y_train_aug = np.repeat(y_train, repeats=1000, axis=0) #y_train is simply repeated

shift = np.std(x_test)*0.1
X_test_aug = np.repeat(X_test_emsc, repeats=5000, axis=0)
X_test_aug = dataaugment(X_test_aug, betashift = shift, slopeshift = 0.05, multishift = shift)
y_test_aug = np.repeat(y_test, repeats=5000, axis=0) #y_test is simply repeated


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, Reshape, GaussianNoise, MaxPooling1D
from keras.callbacks import ReduceLROnPlateau

#Hyperparameters for the network
DENSE = 128
DROPOUT = 0.5
C1_K  = 128 #Number of kernels/feature extractors for first layer
C1_S  = 64 #Width of the convolutional mini networks
C2_K  = 64
C2_S  = 32
activation='relu'

input_dim = x_train.shape[1]

#The model
def make_model():
    model = Sequential()
    #Adding a bit of GaussianNoise also works as regularization
    #First two is number of filter + kernel size
    model.add(Dense(128, kernel_initializer='normal',input_dim = input_dim, activation='relu'))
    model.add(Dense(128, kernel_initializer='normal',activation='relu'))
    model.add(Dense(128, kernel_initializer='normal',activation='relu'))
    model.add(Dense(128, kernel_initializer='normal',activation='relu'))
    model.add(Dense(1, kernel_initializer='normal',activation='linear'))

    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

    return model
  
model = make_model()

h = model.fit(X_train_aug, y_train_aug, epochs=50, batch_size=1024, validation_data=(X_test_aug, y_test_aug))

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
files.download('model.h5') 
files.download('model.json') 



Cloning into 'Deep-Chemometrics'...
remote: Enumerating objects: 43, done.
remote: Total 43 (delta 0), reused 0 (delta 0), pack-reused 43
Unpacking objects: 100% (43/43), done.
/content/Deep-Chemometrics


Using TensorFlow backend.
W0814 09:44:47.461953 139666592450432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0814 09:44:47.469232 139666592450432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0814 09:44:47.478594 139666592450432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0814 09:44:47.545030 139666592450432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0814 09:44:47.718768 1396665924

Train on 108000 samples, validate on 140000 samples
Epoch 1/50
108000/108000 [==============================] - 3s 32us/step - loss: 0.6381 - mean_absolute_error: 0.6381 - val_loss: 0.5892 - val_mean_absolute_error: 0.5892
Epoch 2/50
108000/108000 [==============================] - 3s 25us/step - loss: 0.3948 - mean_absolute_error: 0.3948 - val_loss: 0.6164 - val_mean_absolute_error: 0.6164
Epoch 3/50
108000/108000 [==============================] - 3s 25us/step - loss: 0.3169 - mean_absolute_error: 0.3169 - val_loss: 0.5386 - val_mean_absolute_error: 0.5386
Epoch 4/50
108000/108000 [==============================] - 2s 23us/step - loss: 0.2590 - mean_absolute_error: 0.2590 - val_loss: 0.5232 - val_mean_absolute_error: 0.5232
Epoch 5/50
108000/108000 [==============================] - 3s 25us/step - loss: 0.2233 - mean_absolute_error: 0.2233 - val_loss: 0.5759 - val_mean_absolute_error: 0.5759
Epoch 6/50
108000/108000 [==============================] - 3s 25us/step - loss: 0.1837 - mea

MessageError: ignored

In [3]:
np.squeeze(yscaler.inverse_transform(model.predict(X_check_emsc)))

array([58.029102, 74.650276, 61.961594, 73.57998 , 68.29044 , 67.304   ,
       66.07796 , 65.72936 , 63.495   , 50.84944 , 45.497314, 47.78563 ,
       50.859802, 68.13995 , 62.0005  , 50.62425 , 58.09921 , 61.399555,
       68.85865 , 61.731323, 71.67718 , 69.25027 , 77.62508 , 65.57805 ,
       82.34203 , 71.26531 , 71.26531 , 64.81685 , 60.961563, 53.25929 ,
       68.28605 , 79.03267 , 76.17261 , 76.2956  , 83.75631 ],
      dtype=float32)

In [5]:
np.squeeze(yscaler.inverse_transform(y_check))

array([53., 53., 53., 80., 80., 80., 71., 71., 71., 65., 65., 65., 63.,
       63., 63., 55., 55., 55., 82., 82., 82., 65., 65., 65., 73., 73.,
       73., 67., 67., 67., 57., 57., 57., 84., 84.])